# Deep Research
One of the classic cross-business Agentic use cases! 

In [2]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [3]:

load_dotenv(override=True)

True

### OpenAI Hosted Tools
 
 OpenAI Agents SDK includes the following hosted tools:

The WebSearchTool lets an agent search the web.

The FileSearchTool allows retrieving information from your OpenAI Vector Stores.

The ComputerTool allows automating computer use tasks like taking screenshots and clicking.

In [4]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [5]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities:

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents developed through various methods. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Polymorphic Combinatorial Framework (PCF)**: Utilizes LLMs and mathematical frameworks to guide the design of adaptive AI agents. PCF enables real-time parameter reconfiguration through combinatorial spaces, allowing agents to dynamically adapt their behaviors. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

- **GoalfyMax**: A protocol-driven multi-agent system for intelligent experience entities. It introduces a standardized Agent-to-Agent communication layer and incorporates an Experience Pack architecture for structured knowledge retention and continual learning. ([arxiv.org](https://arxiv.org/abs/2507.09497?utm_source=openai))

- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents. It focuses on curating high-quality training data and enhancing agent robustness and performance. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

- **Google Antigravity**: An AI-powered integrated development environment (IDE) that enables developers to delegate complex coding tasks to autonomous AI agents. It introduces an "agent-first" paradigm, shifting from traditional AI code assistance to a system where AI agents operate with greater autonomy. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Google_Antigravity?utm_source=openai))

- **Agent 365**: Microsoft's tool designed to automate tasks, streamline workflows, and operate across different digital environments. It integrates with Microsoft 365 apps, adding AI support to Word, Excel, PowerPoint, and Outlook. ([windowscentral.com](https://www.windowscentral.com/microsoft/microsoft-doubles-down-on-agentic-ai-agent-365-prepares-for-a-future-with-over-1-billion-agents?utm_source=openai))

- **Amazon Bedrock AgentCore**: AWS's platform designed to streamline the development and deployment of advanced AI agents. It offers modular components tailored for scalability, memory management, identity handling, interoperability, debugging, and performance monitoring. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

- **Model Context Protocol (MCP)**: An open standard introduced by Anthropic to standardize the way AI systems integrate and share data with external tools, systems, and data sources. MCP provides a universal interface for reading files, executing functions, and handling contextual prompts. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai))

- **Agentic Process Fabric**: Pegasystems' framework designed to manage and govern AI agents across various systems and tasks. It enables agents, apps, systems, and data to work together, binding agents to pre-defined workflows, assigned tasks, and service level agreements (SLAs). ([en.wikipedia.org](https://en.wikipedia.org/wiki/Pegasystems?utm_source=openai))

These frameworks represent significant advancements in AI agent development, each contributing to the evolution of autonomous and collaborative AI systems. 

# https://platform.openai.com/traces

In [ ]:
# See note above about cost of WebSearchTool
# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [8]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the most recent developments and frameworks in AI agents suitable for 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore current trends, technologies, and tools being utilized for AI agents.', query='AI agent development tools 2025'), WebSearchItem(reason='To identify major players and technologies in the AI agent space in 2025.', query='top AI agent frameworks companies 2025')]


In [9]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("krachary@myastrotech.com") # Change this to your verified email
    to_email = To("kramkrishnaachary01@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return "success"

In [10]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1150e0860>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [11]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [12]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [13]:
# The next 3 functions will plan and execute the search, using planner_agent and search_agent
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [14]:
# The next 2 functions write a report and email it
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [15]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!
